In [33]:
import torch 
import os
from utils.seqlab import preprocessing_kmer
from transformers import BertTokenizer
from tqdm import tqdm

device = "cuda:0"
tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
test_file = os.path.join("workspace", "seqlab-latest", "test.csv")
batch_size = 8
test_dataloader = preprocessing_kmer(test_file, tokenizer, batch_size)
test_size = len(test_dataloader)


Preparing Data test: 100%|██████████| 6961/6961 [00:55<00:00, 125.73it/s]


In [10]:
# somehow I do PCA analysis.
from models import seqlab, pretrained
from transformers import BertForMaskedLM
import os
import json
import torch

m = seqlab.DNABERT_SL(
    BertForMaskedLM.from_pretrained(os.path.join("pretrained", "3-new-12w-0")).bert,
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)
n = pretrained.DNABERT_SL.from_pretrained(
    os.path.join("pretrained", "3-new-12w-0"),
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)

checkpoint = torch.load(
    os.path.join("run", "sso01-adamw-lr5e-5-base.lin1-2w1boplw", "latest", "checkpoint.pth"), 
    map_location="cuda:0") # force to cuda:0 device
m.load_state_dict(checkpoint.get("model"))
n.load_state_dict(checkpoint.get("model"))

Some weights of the model checkpoint at pretrained\3-new-12w-0 were not used when initializing DNABERT_SL: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing DNABERT_SL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DNABERT_SL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DNABERT_SL were not initialized from the model checkpoint at pretrained\3-new-12w-0 and are newly initial

<All keys matched successfully>

In [29]:
n.save_pretrained(os.path.join("pretrained", "dnabert-sl-lin1"))

In [27]:
m.train()
n.train()
str(m.state_dict()) == str(n.state_dict())

True

In [34]:
import torch
from utils.seqlab import id2label
import pandas as pd

m.eval()
# n.eval()
m.to(device)
# n.to(device)
token_ids, target_ids, bert_outputs = [], [], []
for step, batch in tqdm(enumerate(test_dataloader), total=test_size, desc="Testing"):
    input_ids, attn_mask, token_type_ids, target_labels = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        out, out_bert, out_head = m(input_ids, attn_mask)
        for b_input_ids, b_target, b_bert in zip(input_ids, target_labels, out_bert):
            for i, j, k in zip(b_input_ids, b_target, b_bert):
                token_ids.append(i.item())
                target_ids.append(j.item())
                bert_outputs.append(" ".join([str(a) for a in k.tolist()]))

df = pd.DataFrame(data={
    "token_id": token_ids,
    "target_id": target_ids,
    "bert_value": bert_outputs
})
df.head(5)

Testing: 100%|██████████| 871/871 [1:28:01<00:00,  6.06s/it]   


,token_id,target_id,bert_value
0,2,-100,1.148287057876587 1.622141718864441 -1.0630100...
1,29,7,0.2518802285194397 1.3096948862075806 0.932622...
2,38,7,0.2287495732307434 1.3885688781738281 0.312197...
3,10,7,0.41064363718032837 1.1078848838806152 0.12063...
4,28,7,0.8710600137710571 1.133277416229248 0.5390868...


In [35]:
df.to_csv(os.path.join("motif_analysis", "token_analysis", "token_bert_value.csv"), index=False)

In [17]:
from transformers import BertTokenizer, BertForMaskedLM
from utils.seqlab import _process_sequence_and_label, id2label, label2id
from models import seqlab
import json
from tqdm import tqdm
from torch import tensor, no_grad, argmax

# get token prediction from categorized dataframes.
import os
import pandas as pd
path = os.path.join("prediction", "dataframe-F1 Score=1.csv")
df = pd.read_csv(path)
print(df.head(5))

tokenizer = BertTokenizer.from_pretrained(os.path.join("pretrained", "3-new-12w-0"))
model = seqlab.DNABERT_SL(
    BertForMaskedLM.from_pretrained(os.path.join("pretrained", "3-new-12w-0")).bert,
    json.load(open(os.path.join("models", "config", "seqlab", "base.lin1.json"), "r"))
)
model.eval()
device = "cuda:0"
model.to(device)
ss_labels = [
    "iiE",
    "iEE", 
    "EEi", 
    "Eii"
]
ss_label_ids = [label2id[a] for a in ss_labels]
arr_tokens, arr_labels, arr_bert_values, arr_pred_tokens = [], [], [], []
for i, r in tqdm(df.iterrows(), total=df.shape[0], desc="Testing"):
    with no_grad():
        b_input_ids, b_attention_mask, b_token_type_ids, b_label_ids = _process_sequence_and_label(
            r["input_tokens"],
            r["target_tokens"],
            tokenizer
        )
        b_input_ids = tensor([b_input_ids]).to(device)
        b_attention_mask = tensor([b_attention_mask]).to(device)
        b_label_ids = tensor([b_label_ids]).to(device)
    b_out, b_out_bert, b_out_head = model(
        b_input_ids, 
        b_attention_mask
    )
    b_pred_ids = argmax(b_out, 2)
    # print(b_input_ids.shape, b_label_ids.shape, b_out_bert.shape)
    for input_ids, label_ids, out_bert, pred_ids in zip(b_input_ids, b_label_ids, b_out_bert, b_pred_ids):
        for i, j, k, p in zip(input_ids, label_ids, out_bert, pred_ids):
            label_id = j.item()
            pred_id = p.item()
            if label_id >= 0:
                if label_id in ss_label_ids:
                    token_label = id2label[label_id]
                    token = tokenizer.ids_to_tokens[i.item()]
                    pred_token= id2label[p.item()]
                    bert_values_str = [str(a) for a in k.tolist()]
                    arr_tokens.append(token)
                    arr_pred_tokens.append(pred_token)
                    arr_labels.append(token_label)
                    arr_bert_values.append(" ".join(bert_values_str))

ndf = pd.DataFrame(data={
    "token": arr_tokens,
    "target": arr_labels,
    "prediction": arr_pred_tokens,
    "bert_value": arr_bert_values
})
ndf.head(5)


   label_sequence_id                                       input_tokens  \
0                916  ACT CTT TTT TTT TTC TCC CCC CCT CTT TTT TTT TT...   
1               1108  TGT GTT TTA TAG AGA GAA AAT ATT TTC TCA CAG AG...   
2               1117  GTG TGT GTT TTA TAG AGA GAA AAT ATT TTC TCA CA...   
3               1135  TGG GGT GTG TGT GTT TTA TAG AGA GAA AAT ATT TT...   
4               1144  CTG TGG GGT GTG TGT GTT TTA TAG AGA GAA AAT AT...   

                                      prediction_ids  \
0  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 7 ...   
1  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   
2  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   
3  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   
4  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...   

                                          target_ids  step  f1_score-iii  \
0  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 7 ...  2051             1   
1  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  2286  

Testing: 100%|██████████| 45/45 [00:15<00:00,  2.85it/s]


,token,target,prediction,bert_value
0,AGA,iiE,iiE,0.08756396174430847 0.035568974912166595 0.370...
1,GAA,iEE,iiE,0.3126485049724579 -1.088175892829895 -0.56700...
2,AGG,EEi,Eii,-0.1707116961479187 0.4697659909725189 1.07950...
3,GGT,Eii,iiE,-0.8013615608215332 0.10018081218004227 1.5437...
4,AGG,iiE,Eii,-0.11103878915309906 -0.24303054809570312 1.02...


In [18]:
def _compare_(row):
    target = row["target"]
    prediction = row["prediction"]
    if target == prediction:
        return 1
    else:
        return 0

# true_pred_df = ndf[ndf["target"] == ndf["prediction"]]
# print(true_pred_df.shape) 2, 4

ndf["comparison"] = ndf.apply(lambda x: _compare_(x), axis=1)
ndf[ndf["comparison"] == 1]

,token,target,prediction,bert_value,comparison
0,AGA,iiE,iiE,0.08756396174430847 0.035568974912166595 0.370...,1
80,AGA,iiE,iiE,0.7666749358177185 -0.15378955006599426 1.4997...,1
81,GAA,iEE,iEE,0.26655879616737366 -1.0910930633544922 -0.665...,1


In [20]:
ndf[ndf["comparison"] == 0]

,token,target,prediction,bert_value,comparison
1,GAA,iEE,iiE,0.3126485049724579 -1.088175892829895 -0.56700...,0
2,AGG,EEi,Eii,-0.1707116961479187 0.4697659909725189 1.07950...,0
3,GGT,Eii,iiE,-0.8013615608215332 0.10018081218004227 1.5437...,0
4,AGG,iiE,Eii,-0.11103878915309906 -0.24303054809570312 1.02...,0
5,GGA,iEE,iiE,-0.06882086396217346 0.0999293401837349 1.1488...,0
...,...,...,...,...,...
175,GGT,Eii,iiE,-0.12136223912239075 0.24540439248085022 1.074...,0
176,AGG,iiE,Eii,0.19183388352394104 -0.3378389775753021 0.5174...,0
177,GGA,iEE,EEE,0.17814867198467255 -0.09485137462615967 1.077...,0
178,AGG,EEi,EEE,0.021793914958834648 0.720974862575531 0.61671...,0


In [19]:
ndf.to_csv(
    os.path.join("motif_analysis", "token_analysis", "df-F1=1.csv"), 
    index=False
)

In [21]:
# something weird.
import os
import pandas as pd

path = os.path.join("prediction", "dataframe-F1 Score=1.csv")
tokens, predictions, targets = [], [], []
df = pd.read_csv(path)
for step, r in df.iterrows():
    input_tokens = r["input_tokens"].split(" ")
    prediction_tokens = r["prediction_tokens"].split(" ")
    target_tokens = r["target_tokens"].split(" ")

    for i, j, k in zip(input_tokens, prediction_tokens, target_tokens):
        tokens.append(i)
        predictions.append(j)
        targets.append(k)

df = pd.DataFrame(data={
    "token": tokens,
    "prediction": predictions,
    "target": targets
})
df.head(5)

,token,prediction,target
0,ACT,iii,iii
1,CTT,iii,iii
2,TTT,iii,iii
3,TTT,iii,iii
4,TTC,iii,iii


In [23]:
print(df.shape)
print(df[df["prediction"] == df["target"]].shape)
print(df[df["prediction"] != df["target"]].shape)

(22950, 3)
(22950, 3)
(0, 3)
